In [141]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from scipy.sparse import *
from scipy.sparse.linalg import svds
from tqdm import tqdm
from collections import Counter
from itertools import chain, combinations

In [142]:
#플레이 리스트
train = pd.read_json('data/train.json')
#곡 정보
song_meta = pd.read_json('data/song_meta.json')

In [143]:
display(train.head(1))
display(song_meta.head(1))

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0


## song_meta에 노래 빈도수 추가
- 추천리스트에 많은 노래가 나왔을 때 추천 기준으로 삼을 수 있다.
- popularity 컬럼으로 추가해준다.

In [144]:
#모든 노래를 하나의 리스트로 모은다
all_songs = np.concatenate(train['songs'])
#노래의 빈도수를 센다.
song_counts = dict(Counter(all_songs))

#popularity에 넣는다.
song_meta['popularity'] = song_meta['id'].apply(lambda x : song_counts.get(x) if song_counts.get(x) else 0)

# 희소행렬 만들기
- 각 플레이리스트에 달린 태그 유무를 희소행렬로 표현한다.

In [145]:
#각 플레이리스트 보유 태그 수
train['tag_cnt']= train['tags'].apply(lambda x : len(x))

#중복을 제외한 태그 셋
all_tags_set = set(np.concatenate(train['tags']))

#태그를 list형으로 모은다.
tag_lists = train['tags'].tolist()

In [146]:
#태그 아이디 만들기
id_to_tag = dict(zip(range(len(all_tags_set)), all_tags_set))
tag_to_id = dict(zip(all_tags_set, range(len(all_tags_set)) ))

#태그를 할당된 id로 바꾼 list
id_tag_lists = [list(map(lambda x: tag_to_id[x] , tags)) for tags in train['tags']]

In [147]:
row = np.repeat(np.arange(len(train)), train['tag_cnt'].tolist())
col = np.concatenate(id_tag_lists)
data = np.ones(col.shape[0])


#희소 행렬 만들기
ply_tag = csr_matrix((data, (row, col)))
tag_ply = ply_tag.T

# 태그 입력, 곡의 빈도수 기반 추천 - 세부과정
1. **사용자로부터 태그를 입력받는다.** -> ***`input_tag_id`***  
    - `len(tag_cnt) >= 2`
    - `len(tag_cnt) == 1`
2. **각 태그가 속한 플레이리스트의 곡들을 가져온다.**  
    - `플레이리스트 곡들의 총 합이 30곡 보다 작으면 중복제거 후 바로 제공` - *다음에*
3. **각 태그의 전체 곡들을 빈도수로 정렬하고 상위 n개로 자른다.** -> ***`popular_songs_each_tag`***
4. **각 곡모음 모든 조합의 교집합을 구한다.**
    -  **?** 교집합 결과가 30곡 미만인 경우 - *다음에*
5. **결과 노래를 그대로 제공한다 (상위 노래일수록 많은 플레이리스에서 등장한 노래) or  
  ~~노래 `popularity`로 정렬하여 제공한다.~~**

### **태그 입력받기**

In [148]:
input_tags = ['비오는날','이별','감성','저녁']

# is not None 조건으로 존재하지 않는 태그의 경우 반영하지 않는다.
input_tag_id = [tag_to_id.get(x) for x in input_tags if tag_to_id.get(x) is not None]
input_tag_id

[4106, 219, 1034, 2880]

### **각 태그가 속한 플레이리스트 가져오기**

In [149]:
selected_playlists = []
for tag_id in input_tag_id:
    # tag_id인 열 가져오기
    temp_list = ply_tag[:,tag_id].toarray().reshape(-1)
    # 그 중 1인 것만 playlist에 넣기
    selected_playlists.append(np.argwhere(temp_list == 1).reshape(-1))

selected_playlists

[array([    18,     75,    118, ..., 114977, 115008, 115011], dtype=int64),
 array([     6,     45,     55, ..., 115025, 115045, 115052], dtype=int64),
 array([    16,     24,     42, ..., 115026, 115030, 115039], dtype=int64),
 array([   198,    295,    338, ..., 114738, 114767, 114822], dtype=int64)]

### **각 태그별 플레이리스트들의 노래를 하나의 리스트들로 만들고 빈도를 센다.**
- 중복이 존재할 수 있다.

In [150]:
tag_playlist_songs = []
for playlist in selected_playlists:
    # 태그에 딸린 플레이리스트들의 곡을 합친 리스트.
    temp = np.concatenate(train.iloc[playlist]['songs'].tolist())
    tag_playlist_songs.append(temp)

# 각 노래리스트에서 빈도수를 센다. 
counts = []
for song in tag_playlist_songs:
    counts.append(dict(Counter(song)))

### **각 노래들의 빈도 순위로 자르기**

In [151]:
popular_songs_each_tag = []
for count in counts:
    popular_songs_each_tag.append(sorted(count, key= count.get, reverse = True)[:200])

> `popular_songs_each_tag` - 각 태그가 붙은 플레이리스트에서 등장한 상위 500곡

### **모든 조합의 교집합 구하기**

In [152]:
input_tag_cnt = len(input_tag_id)

# 모든 조합 구하기
combs = list(chain.from_iterable(combinations(popular_songs_each_tag, r) for r in range(2, (input_tag_cnt + 1))))

In [153]:
#조합을 순회하며 겹치는 노래들을 추천 플레이 리스트에 추가
#밑에 있을수록 다수의 플레이리스트에서 겹치는 노래다. -> reverse로 반대로 만든다.
rec_playlist = []
for comb in combs:
    rec_playlist.extend(list(set.intersection(*map(set,comb))))
rec_playlist.reverse()

print(f'중복 제거 전 곡 수 :{len(rec_playlist)}')

# 혹시 있을 중복을 제거한다.
# 순서를 유지하기 위해 df로 처리한다.
rec_df = pd.DataFrame(rec_playlist)
rec_df.drop_duplicates(inplace= True, ignore_index = True)

print(f'최종 결과 곡 수 :{len(rec_df)}')

중복 제거 전 곡 수 :921
최종 결과 곡 수 :194


### 최종 결과 곡들의 상위 30곡

In [154]:
song_meta.iloc[rec_df[0]][['song_name','artist_name_basket','popularity']].head(30)

,song_name,artist_name_basket,popularity
547967,불안해,[혜지 (Hyeji)],1434
235773,이 노래를 듣게 된다면 (Feat. 이소진),[어쿠스틱 멜로디 (Acoustic Melody)],1320
68348,손편지 (Vocal 태인),[아재],1324
703096,사랑에 연습이 있었다면 (Prod. 2soo),[임재현],837
357367,비,[폴킴],1981
38261,한번쯤 (With 박은옥),[아재],1060
448116,MOM (겨울나무),[홍아],1368
215411,지금보다 조금 (Feat. 이원),[어쿠스틱 멜로디 (Acoustic Melody)],1408
674160,늦은 밤 너의 집 앞 골목길에서,[노을],1119
209135,소란했던 시절에,[빌리어코스티],1312


### ~~최종 결과 곡들을 전체인기도로 정렬 후 상위 30곡~~
- 아무데나 들어가는 국밥곡들이 있어서 이렇게는 안됨

In [157]:
song_meta.iloc[rec_df[0]][['song_name','artist_name_basket','popularity']].sort_values(by = 'popularity', ascending = False).head(30)

,song_name,artist_name_basket,popularity
144663,밤편지,[아이유],2175
116573,안아줘,[정준일],2121
357367,비,[폴킴],1981
366786,가끔 미치도록 네가 안고 싶어질 때가 있어,[가을방학],1919
349492,어떤이별,[임승부],1599
675115,야생화,[박효신],1598
463173,비가 내렸어 (Vocal by 스티브언니),[업라이트 (Upright)],1544
42155,벙어리,[홍아],1540
396828,쉬운사랑,[중신],1538
461341,어떻게 지내,[Crush],1526


---

# 테스트 해보기

In [163]:
def recommend(input_tags):  
    
    input_tag_id = [tag_to_id.get(x) for x in input_tags if tag_to_id.get(x) is not None]
    if not input_tag_id:
        print('입력하신 태그가 모두 존재하지 않습니다.')
        return 0
    
    selected_playlists = []
    for tag_id in input_tag_id:
        # tag_id인 열 가져오기
        temp_list = ply_tag[:,tag_id].toarray().reshape(-1)
        # 그 중 1인 것만 playlist에 넣기
        selected_playlists.append(np.argwhere(temp_list == 1).reshape(-1))

    tag_playlist_songs = []
    for playlist in selected_playlists:
        # 태그에 딸린 플레이리스트들의 곡을 합친 리스트.
        temp = np.concatenate(train.iloc[playlist]['songs'].tolist())
        tag_playlist_songs.append(temp)

    counts = []
    for song in tag_playlist_songs:
        counts.append(dict(Counter(song)))
        
    popular_songs_each_tag = []
    for count in counts:
        popular_songs_each_tag.append(sorted(count, key= count.get, reverse = True)[:200])
    
    
    # 인풋 태그가 한개면 그냥 상위 30곡 리턴
    input_tag_cnt = len(input_tag_id)
    if input_tag_cnt < 2:
        return song_meta.iloc[popular_songs_each_tag[0]][['song_name','artist_name_basket','popularity']].head(30)
    
    
    # 모든 조합 구하기
    combs = list(chain.from_iterable(combinations(popular_songs_each_tag, r) for r in range(2, (input_tag_cnt + 1))))

    rec_playlist = []
    for comb in combs:
        rec_playlist.extend(list(set.intersection(*map(set,comb))))
    rec_playlist.reverse()
        
    
    rec_df = pd.DataFrame(rec_playlist)
    rec_df.drop_duplicates(inplace= True, ignore_index = True)
    
    if len(rec_df) == 0:
        print('곡이 존재하지 않습니다.')
        return 0
    
    return song_meta.iloc[rec_df[0]][['song_name','artist_name_basket','popularity']].head(30)

### 테스트 하려면 아래 셀을 실행

In [164]:
while True:
    test_tag = list(input('태그를 입력하세요(띄어쓰기 구분) :' ).split(' '))
    if not test_tag or test_tag == ['']:
        print('태그 입력하세요')
        continue
    else:
        r1 = recommend(test_tag)
        display(r1)
    break

태그를 입력하세요(띄어쓰기 구분) : 크리스마스 캐롤 pop 팝 christmas xmas 


,song_name,artist_name_basket,popularity
387859,All I Want For Christmas Is You,[Mariah Carey],788
358186,Santa Tell Me,[Ariana Grande],694
579838,벌써 크리스마스,"[케이윌, 소유 (SOYOU), 보이프렌드, 몬스타엑스, 우주소녀, 유승우, 브라더...",166
185211,Baby It`s Cold Outside,"[Idina Menzel, Michael Buble]",142
667770,Bring Me Love,[John Legend],84
695032,크리스마스니까,"[성시경, 박효신, 이석훈, 서인국, VIXX (빅스)]",839
213493,"Let it Snow, Let it Snow, Let it Snow",[제이레빗(J Rabbit)],84
320500,Cozy Little Christmas,[Katy Perry],36
137969,It`s Beginning To Look A Lot Like Christmas,[Michael Buble],155
539630,Santa Claus Is Coming To Town,[Michael Buble],92
